In [1]:
import pulp
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('max_colwidth', -1)
pd.set_option('display.expand_frame_repr', False)

In [2]:
df1 = pd.read_csv(r"C:\Users\Sapna Tambole\Downloads\ind_sl_t20i_1st_2022_02_24.csv")

In [3]:
df1 = df1.join(pd.get_dummies(df1[['Role', 'Team']]))
df1['PlayingXI'] = 1

In [4]:
df1.head()

,SeriesId,MatchId,Player,PlayerId,Team,TeamId,Role,Sel,Credits,Series Points,PlayingXI,Points,DT,Role_AR,Role_BAT,Role_BOWL,Role_WK,Team_IND,Team_SL
0,3692,38632,I Kishan,10276,IND,2,WK,73.23,8.5,0,1,141,1,0,0,0,1,1,0
1,3692,38632,S Iyer,9428,IND,2,BAT,71.23,8.5,0,1,84,1,0,1,0,0,1,0
2,3692,38632,B Kumar,1726,IND,2,BOWL,22.17,8.5,0,1,76,1,0,0,1,0,1,0
3,3692,38632,C Asalanka,10934,SL,5,BAT,64.60,8.5,0,1,70,1,0,1,0,0,0,1
4,3692,38632,R Sharma,576,IND,2,BAT,87.30,10.5,0,1,66,1,0,1,0,0,1,0


In [5]:
team = list(df1.Team.unique())
team1, team2 = team[0], team[1]

In [6]:
pickup_status = pulp.LpVariable.dicts('pickup_status',((obj) for obj in df1.index), cat = 'Binary' )

mo = pulp.LpProblem('Profit maximisation problem', pulp.LpMaximize)


mo += pulp.lpSum([pickup_status[obj]*df1.loc[(obj),'Sel'] for obj in df1.index])

mo += pulp.lpSum([pickup_status[obj]*df1.loc[(obj),'Credits']] for obj in df1.index) <= 100

mo += pulp.lpSum([pickup_status[obj]*df1.loc[(obj), 'PlayingXI']] for obj in df1.index) == 11

mo += pulp.lpSum([pickup_status[obj]*df1.loc[(obj), f"Team_{team1}"]] for obj in df1.index) <= 7
mo += pulp.lpSum([pickup_status[obj]*df1.loc[(obj), f"Team_{team1}"]] for obj in df1.index) >= 3
mo += pulp.lpSum([pickup_status[obj]*df1.loc[(obj), f"Team_{team2}"]] for obj in df1.index) <= 7
mo += pulp.lpSum([pickup_status[obj]*df1.loc[(obj), f"Team_{team2}"]] for obj in df1.index) >= 3

mo += pulp.lpSum([pickup_status[obj]*df1.loc[(obj), 'Role_WK']] for obj in df1.index) >= 1
mo += pulp.lpSum([pickup_status[obj]*df1.loc[(obj), 'Role_WK']] for obj in df1.index) <= 4

mo += pulp.lpSum([pickup_status[obj]*df1.loc[(obj), 'Role_AR']] for obj in df1.index) >= 1
mo += pulp.lpSum([pickup_status[obj]*df1.loc[(obj), 'Role_AR']] for obj in df1.index) <= 4

mo += pulp.lpSum([pickup_status[obj]*df1.loc[(obj), 'Role_BAT']] for obj in df1.index) >= 1
mo += pulp.lpSum([pickup_status[obj]*df1.loc[(obj), 'Role_BAT']] for obj in df1.index) <= 6

mo += pulp.lpSum([pickup_status[obj]*df1.loc[(obj), 'Role_BOWL']] for obj in df1.index) >= 1
mo += pulp.lpSum([pickup_status[obj]*df1.loc[(obj), 'Role_BOWL']] for obj in df1.index) <= 6

mo.solve()

df1['pickup_status'] = 0
for obj in df1.index:
    if pickup_status[obj].varValue:
        df1['pickup_status'][obj] = 1
        

fdf1 = df1[df1['pickup_status'] == 1].sort_values(['Role_BAT', 'Role_WK', 'Role_AR', 'Role_BOWL'], ascending = False).reset_index(drop = True)

In [7]:
fdf1[['Player', 'PlayerId', 'Team', 'Role', 'Sel', 'Credits', 'Series Points', 'PlayingXI', 'pickup_status', 'Points']]

,Player,PlayerId,Team,Role,Sel,Credits,Series Points,PlayingXI,pickup_status,Points
0,S Iyer,9428,IND,BAT,71.23,8.5,0,1,1,84
1,C Asalanka,10934,SL,BAT,64.60,8.5,0,1,1,70
2,R Sharma,576,IND,BAT,87.30,10.5,0,1,1,66
3,P Nissanka,13682,SL,BAT,85.62,9.5,0,1,1,2
4,I Kishan,10276,IND,WK,73.23,8.5,0,1,1,141
5,K Mendis,9494,SL,WK,53.58,9.0,0,1,1,0
6,R Jadeja,587,IND,AR,74.73,9.0,0,1,1,34
7,W Hasaranga,10926,SL,AR,69.76,9.5,0,1,1,0
8,D Chameera,8393,SL,BOWL,78.22,9.0,0,1,1,30
9,H Patel,8175,IND,BOWL,65.20,8.5,0,1,1,8


In [29]:
p1 = fdf1.Points.sum()

In [9]:
df2 = pd.read_csv(r"C:\Users\Sapna Tambole\Downloads\ind_sl_t20i_2nd_2022_02_26.csv")

In [10]:
df2 = df2.join(pd.get_dummies(df2[['Role', 'Team']]))
df2['PlayingXI'] = 1

In [11]:
df2.head()

,SeriesId,MatchId,Player,PlayerId,Team,TeamId,Role,Sel,Credits,Series Points,Series DT,PlayingXI,Points,DT,Role_AR,Role_BAT,Role_BOWL,Role_WK,Team_IND,Team_SL
0,3692,38637,S Iyer,9428,IND,2,BAT,87.64,8.5,84,1,1,112,1,0,1,0,0,1,0
1,3692,38637,P Nissanka,13682,SL,5,BAT,66.77,9.5,2,0,1,100,1,0,1,0,0,0,1
2,3692,38637,R Jadeja,587,IND,2,AR,76.24,9.0,34,1,1,93,1,1,0,0,0,1,0
3,3692,38637,D Shanaka,8422,SL,5,AR,68.26,9.0,32,1,1,77,1,1,0,0,0,0,1
4,3692,38637,S Samson,8271,IND,2,WK,25.94,8.0,0,0,1,59,1,0,0,0,1,1,0


In [12]:
pickup_status = pulp.LpVariable.dicts('pickup_status',((obj) for obj in df2.index), cat = 'Binary')

mo = pulp.LpProblem('Profit Maximization Problem', pulp.LpMaximize)

mo += pulp.lpSum([pickup_status[obj]*df2.loc[(obj), 'Sel'] for obj in df2.index])

mo += pulp.lpSum([pickup_status[obj]*df2.loc[(obj), 'PlayingXI'] for obj in df2.index]) == 11

mo += pulp.lpSum([pickup_status[obj]*df2.loc[(obj), 'Credits'] for obj in df2.index]) <= 100

mo += pulp.lpSum([pickup_status[obj]*df2.loc[(obj), 'Team_IND'] for obj in df2.index]) <= 7
mo += pulp.lpSum([pickup_status[obj]*df2.loc[(obj), 'Team_IND'] for obj in df2.index]) >= 3

mo += pulp.lpSum([pickup_status[obj]*df2.loc[(obj), 'Team_SL'] for obj in df2.index]) <= 7
mo += pulp.lpSum([pickup_status[obj]*df2.loc[(obj), 'Team_SL'] for obj in df2.index]) >= 3

mo += pulp.lpSum([pickup_status[obj]*df2.loc[(obj), 'Role_AR'] for obj in df2.index]) <= 4
mo += pulp.lpSum([pickup_status[obj]*df2.loc[(obj), 'Role_AR'] for obj in df2.index]) >= 1

mo += pulp.lpSum([pickup_status[obj]*df2.loc[(obj), 'Role_WK'] for obj in df2.index]) <= 4
mo += pulp.lpSum([pickup_status[obj]*df2.loc[(obj), 'Role_WK'] for obj in df2.index]) >= 1

mo += pulp.lpSum([pickup_status[obj]*df2.loc[(obj), 'Role_BAT'] for obj in df2.index]) <= 6
mo += pulp.lpSum([pickup_status[obj]*df2.loc[(obj), 'Role_BAT'] for obj in df2.index]) >= 1

mo += pulp.lpSum([pickup_status[obj]*df2.loc[(obj), 'Role_BOWL'] for obj in df2.index]) <= 6
mo += pulp.lpSum([pickup_status[obj]*df2.loc[(obj), 'Role_BOWL'] for obj in df2.index]) >= 1

mo.solve()

df2['pickup_status'] = 0
for obj in df2.index:
    if pickup_status[obj].varValue:
        df2['pickup_status'][obj] = 1
        
fdf2 = df2[df2['pickup_status'] == 1].sort_values(['Role_AR', 'Role_WK', 'Role_BAT', 'Role_BOWL']).reset_index(drop = True)

In [13]:
fdf2[['Player', 'PlayerId', 'Team', 'Role', 'Sel', 'Credits', 'Series Points', 'PlayingXI', 'pickup_status', 'Points']]

,Player,PlayerId,Team,Role,Sel,Credits,Series Points,PlayingXI,pickup_status,Points
0,B Kumar,1726,IND,BOWL,63.78,8.5,76,1,1,37
1,D Chameera,8393,SL,BOWL,72.85,9.0,30,1,1,31
2,S Iyer,9428,IND,BAT,87.64,8.5,84,1,1,112
3,P Nissanka,13682,SL,BAT,66.77,9.5,2,1,1,100
4,R Sharma,576,IND,BAT,87.27,10.5,66,1,1,13
5,C Asalanka,10934,SL,BAT,77.97,8.5,70,1,1,6
6,I Kishan,10276,IND,WK,83.71,8.5,141,1,1,22
7,R Jadeja,587,IND,AR,76.24,9.0,34,1,1,93
8,D Shanaka,8422,SL,AR,68.26,9.0,32,1,1,77
9,V Iyer,10917,IND,AR,65.32,8.5,50,1,1,12


In [28]:
p2 = fdf2.Points.sum()

In [15]:
df3 = pd.read_csv(r"C:\Users\Sapna Tambole\Downloads\ind_sl_t20i_3rd_2022_02_27.csv")

In [16]:
df3 = df3.join(pd.get_dummies(df3[['Role', 'Team']]))
df3['PlayingXI'] = 1

In [18]:
df3.head()

,SeriesId,MatchId,Player,PlayerId,Team,TeamId,Role,Sel,Credits,Series Points,Series DT,PlayingXI,Points,Role_AR,Role_BAT,Role_BOWL,Role_WK,Team_IND,Team_SL
0,3692,38642,S Iyer,9428,IND,2,BAT,88.8,8.5,196.0,2,1,100,0,1,0,0,1,0
1,3692,38642,R Jadeja,587,IND,2,AR,87.6,9.0,127.0,2,1,31,1,0,0,0,1,0
2,3692,38642,S Samson,8271,IND,2,WK,85.5,8.0,59.0,1,1,35,0,0,0,1,1,0
3,3692,38642,D Shanaka,8422,SL,5,AR,82.3,9.0,109.0,2,1,105,1,0,0,0,0,1
4,3692,38642,P Nissanka,13682,SL,5,BAT,82.0,9.5,102.0,1,1,-1,0,1,0,0,0,1


In [21]:
pickup_status = pulp.LpVariable.dicts('pickup_status',((obj) for obj in df3.index), cat = 'Binary')

mo = pulp.LpProblem('Profit Maximization Problem', pulp.LpMaximize)

mo += pulp.lpSum([pickup_status[obj]*df3.loc[(obj), 'Sel'] for obj in df3.index])

mo += pulp.lpSum([pickup_status[obj]*df3.loc[(obj), 'PlayingXI'] for obj in df3.index]) == 11

mo += pulp.lpSum([pickup_status[obj]*df3.loc[(obj), 'Credits'] for obj in df3.index]) <= 100

mo += pulp.lpSum([pickup_status[obj]*df3.loc[(obj), 'Team_IND'] for obj in df3.index]) <= 7
mo += pulp.lpSum([pickup_status[obj]*df3.loc[(obj), 'Team_IND'] for obj in df3.index]) >= 3

mo += pulp.lpSum([pickup_status[obj]*df3.loc[(obj), 'Team_SL'] for obj in df3.index]) <= 7
mo += pulp.lpSum([pickup_status[obj]*df3.loc[(obj), 'Team_SL'] for obj in df3.index]) >= 3

mo += pulp.lpSum([pickup_status[obj]*df3.loc[(obj), 'Role_AR'] for obj in df3.index]) <= 4
mo += pulp.lpSum([pickup_status[obj]*df3.loc[(obj), 'Role_AR'] for obj in df3.index]) >= 1

mo += pulp.lpSum([pickup_status[obj]*df3.loc[(obj), 'Role_WK'] for obj in df3.index]) <= 4
mo += pulp.lpSum([pickup_status[obj]*df3.loc[(obj), 'Role_WK'] for obj in df3.index]) >= 1

mo += pulp.lpSum([pickup_status[obj]*df3.loc[(obj), 'Role_BAT'] for obj in df3.index]) <= 6
mo += pulp.lpSum([pickup_status[obj]*df3.loc[(obj), 'Role_BAT'] for obj in df3.index]) >= 1

mo += pulp.lpSum([pickup_status[obj]*df3.loc[(obj), 'Role_BOWL'] for obj in df3.index]) <= 6
mo += pulp.lpSum([pickup_status[obj]*df3.loc[(obj), 'Role_BOWL'] for obj in df3.index]) >= 1

mo.solve()

df3['pickup_status'] = 0
for obj in df3.index:
    if pickup_status[obj].varValue:
        df3['pickup_status'][obj] = 1
        
fdf3 = df3[df3['pickup_status'] == 1].sort_values(['Role_AR', 'Role_WK', 'Role_BAT', 'Role_BOWL']).reset_index(drop = True)

In [24]:
fdf3[['Player', 'PlayerId', 'Team', 'Role', 'Sel', 'Credits', 'Series Points', 'PlayingXI', 'pickup_status', 'Points']]

,Player,PlayerId,Team,Role,Sel,Credits,Series Points,PlayingXI,pickup_status,Points
0,D Chameera,8393,SL,BOWL,65.41,9.0,61.0,1,1,31
1,L Kumara,10928,SL,BOWL,60.44,8.5,87.0,1,1,60
2,H Patel,8175,IND,BOWL,57.31,8.5,31.0,1,1,29
3,S Iyer,9428,IND,BAT,88.80,8.5,196.0,1,1,100
4,P Nissanka,13682,SL,BAT,82.00,9.5,102.0,1,1,-1
5,R Sharma,576,IND,BAT,81.45,10.5,78.2,1,1,10
6,D Gunathilaka,8387,SL,BAT,60.51,8.5,56.0,1,1,2
7,C Asalanka,10934,SL,BAT,57.96,8.5,76.0,1,1,9
8,S Samson,8271,IND,WK,85.50,8.0,59.0,1,1,35
9,R Jadeja,587,IND,AR,87.60,9.0,127.0,1,1,31


In [30]:
p3 = fdf3.Points.sum()

In [31]:
# Average Fantasy Point Per Match
(p1+p2+p3)/3

451.6666666666667